In [1]:
from sklearn.decomposition import PCA, LatentDirichletAllocation, NMF, FactorAnalysis
from sklearn.cluster import KMeans, SpectralClustering, Birch
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture
from sklearn.covariance import GraphicalLasso
from sklearn.metrics import mean_squared_error, silhouette_score
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from sklearn.impute import KNNImputer

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle
from time import time

from collections import defaultdict
from scipy.stats import hmean
from scipy.spatial.distance import cdist
from scipy import stats
import numbers

In [3]:
def weighted_hamming(data):
    categories_dist = []

    for category in data:
        print(f'categories: {category}')
        X = pd.get_dummies(data[category])  #applying one-hot encoding - categorical value to binary vectors
        X_mean = X * X.mean() #weighted binary values
        X_dot = X_mean.dot(X.transpose()) #calculating dot product - similarity btw different rows
        X_np = np.asarray(X_dot.replace(0,1,inplace=False))
        categories_dist.append(X_np)  #store all similarity matrices for each categorical variable
    categories_dist = np.array(categories_dist)
    distances = hmean(categories_dist, axis=0)  #computing harmonic mean - single weighted hamming distance matrix (sensitive to dissimilarities)
    return distances

In [4]:
# Distance Matrix Function
def distance_matrix(data, numeric_distance="euclidean", categorical_distance="jaccard"):
    is_numeric = [all(isinstance(n, numbers.Number) for n in data.iloc[:, i]) for i, x in enumerate(data)]
    is_all_numeric = sum(is_numeric) == len(is_numeric)
    is_all_categorical = sum(is_numeric) == 0
    is_mixed_type = not is_all_categorical and not is_all_numeric

    if is_mixed_type:
        data_numeric = data.iloc[:, is_numeric]
        data_numeric = (data_numeric - data_numeric.mean()) / (data_numeric.max() - data_numeric.min())
        data_categorical = data.iloc[:, [not x for x in is_numeric]]

    if is_mixed_type:
        data_numeric.fillna(data_numeric.mean(), inplace=True)
        for x in data_categorical:
            data_categorical[x].fillna(data_categorical[x].mode()[0], inplace=True)
    elif is_all_numeric:
        data.fillna(data.mean(), inplace=True)
    else:
        for x in data:
            data[x].fillna(data[x].mode()[0])

    if is_all_numeric:
        result_matrix = cdist(data, data, metric=numeric_distance)
    elif is_all_categorical:
        if categorical_distance == "weighted-hamming":
            result_matrix = weighted_hamming(data)
        else:
            result_matrix = cdist(data, data, metric=categorical_distance)
    else:
        result_numeric = cdist(data_numeric, data_numeric, metric=numeric_distance)
        if categorical_distance == "weighted-hamming":
            result_categorical = weighted_hamming(data_categorical)
        else:
            result_categorical = cdist(data_categorical, data_categorical, metric=categorical_distance)
        result_matrix = np.array([[1.0 * (result_numeric[i, j] * len(data_numeric.columns) + result_categorical[i, j] *
                                len(data_categorical.columns)) / len(data.columns) for j in range(len(data))] for i in range(len(data))])

    np.fill_diagonal(result_matrix, np.nan)

    return pd.DataFrame(result_matrix)

In [5]:
# KNN Imputation with Distance Matrix
def knn_impute(data, numeric_distance="euclidean", categorical_distance="jaccard", k_neighbors=1):
    distances = distance_matrix(data, numeric_distance, categorical_distance)
    for column in data.columns:
        if data[column].isnull().any():
            for i in data[column][data[column].isnull()].index:
                nearest_neighbors = distances.loc[i].nsmallest(k_neighbors + 1).iloc[1:].index
                if pd.api.types.is_numeric_dtype(data[column]):
                    data.at[i, column] = data[column].loc[nearest_neighbors].mean()
                else:
                    data.at[i, column] = data[column].loc[nearest_neighbors].mode()[0]
    return data

In [ ]:
# Preprocessing Pipeline
with open('/Users/saikrishnakathika/Documents/DATA_245_Machine_Learning/project/code/data.pickle', 'rb') as handle:
    subs_train, subs_test, subs_val, hlth_train, hlth_test, hlth_val, demo_train, demo_test, demo_val = pickle.load(handle)

# Combine data subsets for imputation
all_train = pd.concat([subs_train, hlth_train, demo_train], axis=1)
all_test = pd.concat([subs_test, hlth_test, demo_test], axis=1)
all_val = pd.concat([subs_val, hlth_val, demo_val], axis=1)

# Combine all_test with all_val
all_test = pd.concat([all_test, all_val], axis=0)

# Drop irrelevant columns
all_train = all_train.drop(['QUESTID2', 'FILEDATE'], axis=1, errors='ignore')
all_test = all_test.drop(['QUESTID2', 'FILEDATE'], axis=1, errors='ignore')

subs_train = subs_train.drop(['QUESTID2', 'FILEDATE'], axis=1, errors='ignore')
subs_test = subs_test.drop(['QUESTID2', 'FILEDATE'], axis=1, errors='ignore')
subs_val = subs_val.drop(['QUESTID2', 'FILEDATE'], axis=1, errors='ignore')

# Drop columns with specific patterns (e.g., regex 'II')
all_train = all_train.drop(all_train.filter(regex='II').columns, axis=1)
all_test = all_test.drop(all_test.filter(regex='II').columns, axis=1)

subs_train = subs_train.drop(subs_train.filter(regex='II').columns, axis=1)
hlth_train = hlth_train.drop(hlth_train.filter(regex='II').columns, axis=1)
demo_train = demo_train.drop(demo_train.filter(regex='II').columns, axis=1)

subs_test = subs_test.drop(subs_test.filter(regex='II').columns, axis=1)
hlth_test = hlth_test.drop(hlth_test.filter(regex='II').columns, axis=1)
demo_test = demo_test.drop(demo_test.filter(regex='II').columns, axis=1)

subs_val = subs_val.drop(subs_val.filter(regex='II').columns, axis=1)
hlth_val = hlth_val.drop(hlth_val.filter(regex='II').columns, axis=1)
demo_val = demo_val.drop(demo_val.filter(regex='II').columns, axis=1)

# Replace invalid values (>=80) with NaN
all_train[all_train >= 80] = np.nan
all_test[all_test >= 80] = np.nan

subs_train[subs_train >= 80] = np.nan
hlth_train[hlth_train >= 80] = np.nan
demo_train[demo_train >= 80] = np.nan

subs_test[subs_test >= 80] = np.nan
hlth_test[hlth_test >= 80] = np.nan
demo_test[demo_test >= 80] = np.nan

subs_val[subs_val >= 80] = np.nan
hlth_val[hlth_val >= 80] = np.nan
demo_val[demo_val >= 80] = np.nan

# Drop columns with >80 missing values
all_train = all_train.dropna(axis=1, thresh=int(all_train.shape[0] / 5))
cols1 = [c for c in all_train.columns]
all_test = all_test[cols1]

subs_train = subs_train.dropna(axis=1, thresh=int(subs_train.shape[0] / 5))
cols2 = [c for c in subs_train.columns]
subs_test = subs_test[cols2]
subs_val = subs_val[cols2]

hlth_train = hlth_train.dropna(axis=1, thresh=int(hlth_train.shape[0] / 5))
cols3 = [c for c in hlth_train.columns]
hlth_test = hlth_test[cols3]
hlth_val = hlth_val[cols3]

demo_train = demo_train.dropna(axis=1, thresh=int(demo_train.shape[0] / 5))
cols4 = [c for c in demo_train.columns]
demo_test = demo_test[cols4]
demo_val = demo_val[cols4]

In [7]:
# Apply KNN Imputation on all subsets
all_train = knn_impute(all_train, k_neighbors=1)
all_test = knn_impute(all_test, k_neighbors=1)

subs_train = knn_impute(subs_train, k_neighbors=1)
subs_test = knn_impute(subs_test, k_neighbors=1)
subs_val = knn_impute(subs_val, k_neighbors=1)

hlth_train = knn_impute(hlth_train, k_neighbors=1)
hlth_test = knn_impute(hlth_test, k_neighbors=1)
hlth_val = knn_impute(hlth_val, k_neighbors=1)

demo_train = knn_impute(demo_train, k_neighbors=1)
demo_test = knn_impute(demo_test, k_neighbors=1)
demo_val = knn_impute(demo_val, k_neighbors=1)

# Save all datasets to a pickle file
with open('data_removed_updated.pickle', 'wb') as handle:
    pickle.dump([all_train, all_test, subs_train, subs_test, subs_val, hlth_train, hlth_test, hlth_val, demo_train, demo_test, demo_val], handle, protocol=pickle.HIGHEST_PROTOCOL)

In [9]:
subs_train.info()
subs_train.head()

<class 'pandas.core.frame.DataFrame'>
Index: 45020 entries, 44412 to 30140
Data columns (total 35 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   CIGEVER    45020 non-null  int64  
 1   CIGOFRSM   45020 non-null  float64
 2   CIGWILYR   45020 non-null  float64
 3   CIGTRY     45020 non-null  float64
 4   CIGREC     45020 non-null  float64
 5   CIGDLYMO   45020 non-null  float64
 6   CIGAGE     45020 non-null  float64
 7   CIG100LF   45020 non-null  float64
 8   SMKLSSEVR  45020 non-null  float64
 9   CIGAREVR   45020 non-null  float64
 10  CIGARTRY   45020 non-null  float64
 11  CIGARREC   45020 non-null  float64
 12  PIPEVER    45020 non-null  float64
 13  ALCEVER    45020 non-null  float64
 14  ALCTRY     45020 non-null  float64
 15  ALCREC     45020 non-null  float64
 16  ALCYRTOT   45020 non-null  float64
 17  ALBSTWAY   45020 non-null  float64
 18  ALDAYPMO   45020 non-null  float64
 19  ALDAYPWK   45020 non-null  float64
 20  ALCDAYS

,CIGEVER,CIGOFRSM,CIGWILYR,CIGTRY,CIGREC,CIGDLYMO,CIGAGE,CIG100LF,SMKLSSEVR,CIGAREVR,...,MJREC,COCEVER,HEREVER,METHAMEVR,PNRANYLIF,PNRANYREC,TRQANYLIF,STMANYLIF,SEDANYLIF,PNRNMLIF
44412,1,3.83617,3.8709,16.000000,4.000000,2.000000,17.956857,2.000000,2.0,2.0,...,2.377809,2.0,2.0,2.0,1.0,2.000000,2.0,2.0,2.0,2.000000
31424,2,4.00000,4.0000,15.876284,2.745063,2.288681,17.956857,2.120717,2.0,2.0,...,2.377809,2.0,2.0,2.0,2.0,1.479697,2.0,2.0,2.0,2.164689
27959,1,3.83617,3.8709,13.000000,3.000000,1.000000,14.000000,1.000000,1.0,1.0,...,3.000000,1.0,2.0,2.0,2.0,1.479697,2.0,5.0,2.0,2.164689
4062,1,3.83617,3.8709,21.000000,4.000000,2.000000,17.956857,2.000000,2.0,2.0,...,2.377809,2.0,2.0,2.0,2.0,1.479697,2.0,2.0,2.0,2.164689
41259,1,3.83617,3.8709,14.000000,1.000000,5.000000,14.000000,5.000000,2.0,2.0,...,1.000000,2.0,2.0,2.0,2.0,1.479697,2.0,2.0,2.0,2.164689
